<a href="https://colab.research.google.com/github/thiersY/reseau-neurone-simple/blob/main/NN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Un réseau neuronal simple à partir de zéro avec PyTorch et Google Colab

Dans ce tutoriel, nous implémentons un réseau de neurones simple à partir de zéro en utilisant PyTorch.


## À propos

Dans ce tutoriel, nous allons implémenter un réseau de neurones simple à partir de zéro en utilisant PyTorch. L'idée du tutoriel est de vous apprendre les bases de PyTorch et comment il peut être utilisé pour implémenter un réseau de neurones à partir de zéro. Je passerai en revue certaines des fonctionnalités et concepts de base disponibles dans PyTorch qui vous permettront de créer vos propres réseaux de neurones.

Ce didacticiel suppose que vous avez une connaissance préalable du fonctionnement d'un réseau de neurones. Ne t'inquiète pas! Même si vous n'êtes pas si sûr, tout ira bien. Pour les utilisateurs avancés de PyTorch, ce didacticiel peut toujours servir de rappel. Ce tutoriel s'en inspire fortement [Mise en œuvre du réseau de neurones](https://repl.it/talk/announcements/Build-a-Neural-Network-in-Python/5457) codé uniquement en utilisant Numpy. En fait, j'ai essayé de réimplémenter le code en utilisant PyTorch à la place et j'ai ajouté mes propres intuitions et explications. Grâce à [Samay](https://repl.it/@shamdasani) pour son travail phénoménal, j'espère que cela en inspirera beaucoup d'autres comme moi.

Le module `torch` fournit tous les opérateurs **tensor** nécessaires dont vous aurez besoin pour implémenter votre premier réseau de neurones à partir de zéro dans PyTorch. C'est exact! Dans PyTorch, tout est un tenseur, c'est donc la première chose à laquelle vous devrez vous habituer. Importons les bibliothèques dont nous aurons besoin pour ce tutoriel.

In [1]:
import torch
import torch.nn as nn

## Données
Commençons par créer des exemples de données à l'aide de la commande `torch.tensor`. Dans Numpy, cela pourrait être fait avec `np.array`. Les deux fonctions ont le même objectif, mais dans PyTorch, tout est un tenseur par opposition à un vecteur ou à une matrice. Nous définissons les types dans PyTorch à l'aide de la commande `dtype=torch.xxx`.

Dans les données ci-dessous, « X » représente le nombre d'heures étudiées et le temps que les étudiants ont passé à dormir, tandis que « y » représente les notes. La variable `xPredicted` est une entrée unique pour laquelle nous voulons prédire une note en utilisant les paramètres appris par le réseau de neurones. N'oubliez pas que le réseau de neurones veut apprendre un mappage entre « X » et « y », il essaiera donc de deviner à partir de ce qu'il a appris à partir des données de formation.

In [2]:
X = torch.tensor(([2, 9], [1, 5], [3, 6]), dtype=torch.float) # 3 X 2 tensor
y = torch.tensor(([92], [100], [89]), dtype=torch.float) # 3 X 1 tensor
xPredicted = torch.tensor(([4, 8]), dtype=torch.float) # 1 X 2 tensor

Vous pouvez vérifier la taille des tenseurs que nous venons de créer avec la commande `size`. Cela équivaut à la commande `shape` utilisée dans des outils tels que Numpy et Tensorflow. 

In [3]:
print(X.size())
print(y.size())

torch.Size([3, 2])
torch.Size([3, 1])


## Scaling

Ci-dessous, nous effectuons une mise à l'échelle sur les données de l'échantillon. Notez que la fonction `max` renvoie à la fois un tenseur et les indices correspondants. Nous utilisons donc `_` pour capturer les indices que nous n'utiliserons pas ici car nous ne nous intéressons qu'aux valeurs maximales pour effectuer la mise à l'échelle. Parfait! Nos données sont maintenant dans un format très agréable que notre réseau de neurones appréciera plus tard.

In [4]:
# scale units
X_max, _ = torch.max(X, 0)
xPredicted_max, _ = torch.max(xPredicted, 0)

X = torch.div(X, X_max)
xPredicted = torch.div(xPredicted, xPredicted_max)
y = y / 100  # max test score is 100
print(xPredicted)

tensor([0.5000, 1.0000])


Notez qu'il y a deux fonctions `max` et `div` dont je n'ai pas parlé ci-dessus. Ils font exactement ce qu'ils impliquent : 'max' trouve la valeur maximale dans un vecteur... Je veux dire tenseur ; et `div` est fondamentalement une jolie petite fonction pour diviser deux tenseurs.

## Modèle (graphique de calcul)
Une fois les données traitées et au bon format, il ne vous reste plus qu'à définir votre modèle. C'est là que les choses commencent à changer un peu par rapport à la façon dont vous construiriez vos réseaux de neurones en utilisant, par exemple, quelque chose comme Keras ou Tensorflow. Cependant, vous vous rendrez compte rapidement au fur et à mesure que PyTorch ne diffère pas beaucoup des autres outils d'apprentissage en profondeur. En fin de compte, nous construisons un graphique de calcul, qui est utilisé pour dicter comment les données doivent circuler et quel type d'opérations sont effectuées sur ces informations.

À des fins d'illustration, nous construisons le réseau de neurones ou le graphe de calcul suivant :


![alt text](https://drive.google.com/uc?export=view&id=1l-sKpcCJCEUJV1BlAqcVAvLXLpYCInV6)

In [6]:
class Neural_Network(nn.Module):
    def __init__(self, ):
        super(Neural_Network, self).__init__()
        # paramètres
        # TODO: les paramètres peuvent être paramétrés au lieu de les déclarer ici
        self.inputSize = 2
        self.outputSize = 1
        self.hiddenSize = 3
        
        # weights
        self.W1 = torch.randn(self.inputSize, self.hiddenSize) # 3 X 2 tensor
        self.W2 = torch.randn(self.hiddenSize, self.outputSize) # 3 X 1 tensor
        
    def forward(self, X):
        self.z = torch.matmul(X, self.W1) # 3 X 3 ".dot" ne diffuse pas dans PyTorch
        self.z2 = self.sigmoid(self.z) # fonction d'activation
        self.z3 = torch.matmul(self.z2, self.W2)
        o = self.sigmoid(self.z3) # fonction d'activation final
        return o
        
    def sigmoid(self, s):
        return 1 / (1 + torch.exp(-s))
    
    def sigmoidPrime(self, s):
        # dérivé de sigmoïde
        return s * (1 - s)
    
    def backward(self, X, y, o):
        self.o_error = y - o # erreur de sortie
        self.o_delta = self.o_error * self.sigmoidPrime(o) # dérivée de sig à l'erreur
        self.z2_error = torch.matmul(self.o_delta, torch.t(self.W2))
        self.z2_delta = self.z2_error * self.sigmoidPrime(self.z2)
        self.W1 += torch.matmul(torch.t(X), self.z2_delta)
        self.W2 += torch.matmul(torch.t(self.z2), self.o_delta)
        
    def train(self, X, y):
        # passe avant + arrière pour l'entraînement
        o = self.forward(X)
        self.backward(X, y, o)
        
    def saveWeights(self, model):
        # nous utiliserons les fonctions de stockage interne de PyTorch
        torch.save(model, "NN")
        # vous pouvez recharger le modèle avec tous les poids et ainsi de suite avec :
        # torch.load("NN")
        
    def predict(self):
        print ("Predicted data based on trained weights: ")
        print ("Input (scaled): \n" + str(xPredicted))
        print ("Output: \n" + str(self.forward(xPredicted)))
        

Pour les besoins de ce tutoriel, nous n'allons pas parler de maths, c'est pour un autre jour. Je veux juste que vous ayez une idée de ce qu'il faut pour construire un réseau de neurones à partir de zéro en utilisant PyTorch. Décomposons le modèle qui a été déclaré via la classe ci-dessus.

## En-tête de classe
Tout d'abord, nous avons défini notre modèle via une classe car c'est la manière recommandée pour construire le graphe de calcul. L'en-tête de classe contient le nom de la classe `Neural Network` et le paramètre `nn.Module` qui indique essentiellement que nous définissons notre propre réseau de neurones.

```python
class Neural_Network(nn.Module):
```

## Initialisation
L'étape suivante consiste à définir les initialisations ( `def __init__(self,)`) qui seront effectuées lors de la création d'une instance du réseau de neurones personnalisé. Vous pouvez déclarer les paramètres de votre modèle ici, mais généralement, vous déclarerez la structure de votre réseau dans cette section -- la taille des couches cachées, etc. Puisque nous construisons le réseau de neurones à partir de zéro, nous avons explicitement déclaré la taille des matrices de poids : une qui stocke les paramètres de l'entrée à la couche cachée ; et un qui stocke le paramètre de la couche masquée à la couche de sortie. Les deux matrices de poids sont initialisées avec des valeurs choisies au hasard dans une distribution normale via `torch.randn(...)`. Notez que nous n'utilisons pas de biais uniquement pour garder les choses aussi simples que possible.

```python
def __init__(self, ):
    super(Neural_Network, self).__init__()
    # paramètres
    # TODO: les paramètres peuvent être paramétrés au lieu de les déclarer ici
    self.inputSize = 2
    self.outputSize = 1
    self.hiddenSize = 3

    # weights
    self.W1 = torch.randn(self.inputSize, self.hiddenSize) # 3 X 2 tensor
    self.W2 = torch.randn(self.hiddenSize, self.outputSize) # 3 X 1 tensor
```

## La fonction de transfert
La fonction `forward` est l'endroit où toute la magie se produit (voir ci-dessous). C'est là que les données entrent et sont introduites dans le graphe de calcul (c'est-à-dire la structure de réseau neuronal que nous avons construite). Puisque nous construisons un réseau de neurones simple avec une couche cachée, notre fonction de transfert semble très simple :

```python
def forward(self, X):
    self.z = torch.matmul(X, self.W1) 
    self.z2 = self.sigmoid(self.z) # fonction d'activation
    self.z3 = torch.matmul(self.z2, self.W2)
    o = self.sigmoid(self.z3) # fonction d'activation final
    return o
```

La fonction `forward` ci-dessus prend l'entrée `X` et effectue ensuite une multiplication matricielle (`torch.matmul(...)`) avec la première matrice de poids `self.W1`. Ensuite, le résultat est appliqué une fonction d'activation, "sigmoïde". La matrice résultante de l'activation est ensuite multipliée par la deuxième matrice de poids "self.W2". Ensuite, une autre activation est effectuée, ce qui rend la sortie du réseau neuronal ou du graphe de calcul. Le processus que j'ai décrit ci-dessus est simplement ce qu'on appelle une "passe anticipée". Pour que les poids soient optimisés lors de l'entraînement, nous avons besoin d'un algorithme de rétropropagation.

## La fonction arrière
La fonction `backward` contient l'algorithme de rétropropagation, où le but est essentiellement de minimiser la perte par rapport à nos poids. En d'autres termes, les poids doivent être mis à jour de manière à ce que la perte diminue pendant que le réseau de neurones s'entraîne (enfin, c'est ce que nous espérons). Toute cette magie est possible avec l'algorithme de descente de gradient qui est déclaré dans la fonction `backward`. Prenez une minute ou deux pour inspecter ce qui se passe dans le code ci-dessous :

```python
def backward(self, X, y, o):
    self.o_error = y - o # erreur de sortie
    self.o_delta = self.o_error * self.sigmoidPrime(o) 
    self.z2_error = torch.matmul(self.o_delta, torch.t(self.W2))
    self.z2_delta = self.z2_error * self.sigmoidPrime(self.z2)
    self.W1 += torch.matmul(torch.t(X), self.z2_delta)
    self.W2 += torch.matmul(torch.t(self.z2), self.o_delta)
```

Notez que nous effectuons de nombreuses multiplications de matrices avec les opérations de transposition via les opérations `torch.matmul(...)` et `torch.t(...)`, respectivement. Le reste est simplement une descente en gradient - il n'y a rien à faire.

## Entraînement
Il ne reste plus qu'à former le réseau de neurones. Nous créons d'abord une instance du graphe de calcul que nous venons de construire :

```python
NN = Neural_Network()
```

Ensuite, nous formons le modèle pour `1000` tours. Notez que dans PyTorch, `NN(X)` appelle automatiquement la fonction `forward`, il n'est donc pas nécessaire d'appeler explicitement `NN.forward(X)`.

Après avoir obtenu la sortie prévue pour chaque cycle d'entraînement, nous calculons la perte, avec le code suivant :

```python
torch.mean((y - NN(X))**2).detach().item()
```

L'étape suivante consiste à démarrer l'entraînement (avant + arrière) via `NN.train(X, y)`. Après avoir entraîné le réseau de neurones, nous pouvons stocker le modèle et générer la valeur prédite de l'instance unique que nous avons déclarée au début, "xPredicted".

Entraînons-nous!

In [7]:
NN = Neural_Network()
for i in range(1000):  # trains the NN 1,000 times
    if (i % 100) == 0:
        print ("#" + str(i) + " Loss: " + str(torch.mean((y - NN(X))**2).detach().item()))  # mean sum squared loss
    NN.train(X, y)
#NN.saveWeights(NN) # save weights

NN.predict()

print("Finished training!")

#0 Loss: 0.023130903020501137
#100 Loss: 0.004318820778280497
#200 Loss: 0.004086004104465246
#300 Loss: 0.003987605217844248
#400 Loss: 0.003900497453287244
#500 Loss: 0.0038110651075839996
#600 Loss: 0.003716830164194107
#700 Loss: 0.0036171097308397293
#800 Loss: 0.0035115480422973633
#900 Loss: 0.0033999476581811905
Predicted data based on trained weights: 
Input (scaled): 
tensor([0.5000, 1.0000])
Output: 
tensor([0.9539])
Finished training!


La perte ne cesse de diminuer, ce qui signifie que le réseau de neurones apprend quelque chose. C'est ça. Toutes nos félicitations! Vous venez d'apprendre à créer et à former un réseau de neurones à partir de zéro à l'aide de PyTorch. Il y a tellement de choses que vous pouvez faire avec le réseau peu profond que nous venons de mettre en place. Vous pouvez ajouter plus de couches cachées ou essayer d'incorporer les termes de biais pour la pratique. J'aimerais voir ce que vous allez construire à partir d'ici. Contactez-moi sur[Twitter](https://twitter.com/omarsar0) if you have any further questions or leave your comments here. Until next time!

## References:
- [PyTorch nn. Modules](https://pytorch.org/tutorials/beginner/pytorch_with_examples.html#pytorch-custom-nn-modules)
- [Build a Neural Network with Numpy](https://enlight.nyc/neural-network)